<a href="https://colab.research.google.com/github/IkasuAra/producao_pib_nacional/blob/main/prod_pib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pandasql
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id':"1pLSVwaxHMjnBBb4xmJeXxSYVRxFD7PQH"})
downloaded.GetContentFile('Cópia de produção_2019.xlsx')

downloaded = drive.CreateFile({'id':"1--8Nc04ULGUJez1vLWHxTFCO201txdf4"})
downloaded.GetContentFile('PIB_Cepea.xlsx')

In [ ]:
tb_prod = 'https://docs.google.com/spreadsheets/d/1pLSVwaxHMjnBBb4xmJeXxSYVRxFD7PQH/edit?usp=sharing&ouid=109837334966008233855&rtpof=true&sd=true'

tb_pib = 'https://docs.google.com/spreadsheets/d/1--8Nc04ULGUJez1vLWHxTFCO201txdf4/edit?usp=sharing&ouid=109837334966008233855&rtpof=true&sd=true'

In [ ]:
df_prod = pd.read_excel('Cópia de produção_2019.xlsx', header=4)

df_prod.rename(columns = {'Unnamed: 0':'unidade_federacao', '2019':'area_2019', 'Maio':'maio_area', 'junho':'junho_area', 
                          '2019.1':'producao_2019', 'Maio.1':'maio_producao', 'Junho':'junho_producao'}, inplace = True)

df_prod = df_prod.drop(0)

df_prod['area_2019']= df_prod['area_2019'].map(str)
df_prod['maio_area']= df_prod['maio_area'].map(str)
df_prod['junho_area']= df_prod['junho_area'].map(str)
df_prod['producao_2019']= df_prod['producao_2019'].map(str)
df_prod['maio_producao']= df_prod['maio_producao'].map(str)
df_prod['junho_producao']= df_prod['junho_producao'].map(str)

df_prod = df_prod.replace(' ', '', regex=True)

df_prod['area_2019'] = pd.to_numeric(df_prod['area_2019'], errors='coerce')
df_prod['maio_area'] = pd.to_numeric(df_prod['maio_area'], errors='coerce')
df_prod['junho_area'] = pd.to_numeric(df_prod['junho_area'], errors='coerce')
df_prod['producao_2019'] = pd.to_numeric(df_prod['producao_2019'], errors='coerce')
df_prod['maio_producao'] = pd.to_numeric(df_prod['maio_producao'], errors='coerce')
df_prod['junho_producao'] = pd.to_numeric(df_prod['junho_producao'], errors='coerce')

# media da produção de 2019
df_media_prod = pandasql.sqldf("SELECT CAST(AVG(producao_2019) AS FLOAT) FROM df_prod;", globals())

# porcentagem de participacao da regiao centro-oeste na produção do primeiro semestre de 2019
df_prod_centrooeste = pandasql.sqldf("SELECT unidade_federacao, ROUND(SUM(maio_producao + junho_producao) * 100.0 / (SELECT SUM(maio_producao + junho_producao) FROM df_prod), 2) AS prod_centrooeste FROM df_prod WHERE 1=1 AND unidade_federacao = 'GOIÁS' OR unidade_federacao = 'DISTRITOFEDERAL' OR unidade_federacao = 'MATOGROSSO' OR unidade_federacao = 'MATOGROSSODOSUL' GROUP BY unidade_federacao;", globals())
df_prod_centrooeste = pandasql.sqldf("SELECT SUM(prod_centrooeste) AS percent_prod_centrooeste FROM df_prod_centrooeste;", globals())

# estado que possui maior area produtiva
df_max_area_prod = pandasql.sqldf("SELECT unidade_federacao, MAX(producao_2019/area_2019) AS max_area_prod FROM df_prod WHERE 1=1;", globals())

# media producao mensal no estado de sao paulo
df_avg_prod = pandasql.sqldf("SELECT unidade_federacao, ROUND(CAST((producao_2019/12) AS FLOAT), 2) AS media_prod_mensal FROM df_prod WHERE 1=1 AND unidade_federacao = 'SÃOPAULO';", globals())

In [ ]:
df_pib = pd.read_excel('PIB_Cepea.xlsx', header=7)

df_pib = df_pib.drop(columns=['Unnamed: 6', 'Unnamed: 12', 'Unnamed: 18', 'Unnamed: 24', 'Unnamed: 30'])

df_pib.rename(columns = {'Unnamed: 0':'ano', '(A) Insumos':'insumos_agronegocio_renda', '(B) Agropecuária ':'agropecuaria_agronegocio_renda', '(C) Indústria':'industria_agronegocio_renda', '(D) Serviços':'servicos_agronegocio_renda', 'Agronegócio Total (A+B+C+D)':'agronegocio_total_renda',
                         '(A) Insumos.1':'insumos_agricola_renda', '(B) Agropecuária .1':'agropecuaria_agricola_renda', '(C) Indústria.1':'industria_agricola_renda', '(D) Serviços.1':'servicos_agricola_renda', 'Ramo Agrícola (A+B+C+D)':'agricola_total_renda', 
                         '(A) Insumos.2':'insumos_pecuaria_renda', '(B) Agropecuária .2':'agropecuaria_pecuaria_renda', '(C) Indústria.2':'industria_pecuaria_renda', '(D) Serviços.2':'servicos_pecuaria_renda', 'Ramo Pecuário (A+B+C+D)':'pecuaria_total_renda',
                         '(A) Insumos.3':'insumos_agronegocio_pib', '(B) Agropecuária .3':'agropecuaria_agronegocio_pib', '(C) Indústria.3':'industria_agronegocio_pib', '(D) Serviços.3':'servicos_agronegocio_pib', 'Agronegócio Total (A+B+C+D).1':'agronegocio_total_pib',
                         '(A) Insumos.4':'insumos_agricola_pib', '(B) Agropecuária .4':'agropecuaria_agricola_pib', '(C) Indústria.4':'industria_agricola_pib', '(D) Serviços.4':'servicos_agricola_pib', 'Ramo Agrícola (A+B+C+D).1':'agricola_total_pib', 
                         '(A) Insumos.5':'insumos_pecuaria_pib', '(B) Agropecuária .5':'agropecuaria_pecuaria_pib', '(C) Indústria.5':'industria_pecuaria_pib', '(D) Serviços.5':'servicos_pecuaria_pib', 'Ramo Pecuário (A+B+C+D).1':'pecuaria_total_pib'}, inplace = True)

# correlacao entre a producao agricola, pib total do ramo agricola e pecuario de 2019
df_pib['producao_agricola'] = 8943064 

df_prod_pib = pandasql.sqldf("SELECT producao_agricola, agricola_total_pib, pecuaria_total_pib FROM df_pib WHERE ano = '2019';", globals())

df_prod_pib.plot.bar();

# correlacao entre os segmentos de insumos agricola e pecuaria em 2019
df_prod_pib = pandasql.sqldf("SELECT insumos_agricola_pib, insumos_pecuaria_pib FROM df_pib WHERE ano = '2019';", globals())

df_prod_pib.plot.bar();

# correlacao entre os segmentos de agropecuaria agricola e pecuaria em 2019
df_prod_pib = pandasql.sqldf("SELECT agropecuaria_agricola_pib, agropecuaria_pecuaria_pib FROM df_pib WHERE ano = '2019';", globals())

df_prod_pib.plot.bar();

# correlacao entre os segmentos de industria agricola e pecuaria em 2019
df_prod_pib = pandasql.sqldf("SELECT industria_agricola_pib, industria_pecuaria_pib FROM df_pib WHERE ano = '2019';", globals())

df_prod_pib.plot.bar();

# correlacao entre os segmentos de servicos agricola e pecuaria em 2019
df_prod_pib = pandasql.sqldf("SELECT servicos_agricola_pib, servicos_pecuaria_pib FROM df_pib WHERE ano = '2019';", globals())

df_prod_pib.plot.bar();